In [1]:
!pip install mlflow dagshub optuna

In [1]:
import dagshub
dagshub.init(repo_owner='dakshvandanarathi', repo_name='YT-Sentiment-Analyser', mlflow=True)

Accessing as dakshvandanarathi

Initialized MLflow to track repo "dakshvandanarathi/YT-Sentiment-Analyser"

Repository dakshvandanarathi/YT-Sentiment-Analyser initialized!

In [2]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import lightgbm as lgb
from imblearn.over_sampling import ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [21]:
# Set or create an experiment
mlflow.set_experiment("Exp 4 - ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/498f75c0643c4aa2b0e490aca486b20a', creation_time=1729654450594, experiment_id='5', last_update_time=1729654450594, lifecycle_stage='active', name='Exp 4 - ML Algos with HP Tuning', tags={}>

In [3]:
df = pd.read_csv('/content/preprocessed_data.csv').dropna()
df.shape

(36662, 5)

In [4]:
# Remap the class labels from [-1, 0, 1] to [0, 1, 2]
df['category'] = df['category'].map({-1: 0, 0: 1, 1: 2})

In [5]:
# Define a function to vectorize the data using TF-IDF
def vectorize_data(X_train, X_val, X_test, max_features, ngram_range):
    vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
    X_train_vec = vectorizer.fit_transform(X_train['comment']).toarray()
    X_val_vec = vectorizer.transform(X_val['comment']).toarray()
    X_test_vec = vectorizer.transform(X_test['comment']).toarray()

    # Combine additional features
    X_train_combined = np.hstack([X_train_vec, X_train[['word_count', 'char_count', 'avg_word_length']].values])
    X_val_combined = np.hstack([X_val_vec, X_val[['word_count', 'char_count', 'avg_word_length']].values])
    X_test_combined = np.hstack([X_test_vec, X_test[['word_count', 'char_count', 'avg_word_length']].values])

    return X_train_combined, X_val_combined, X_test_combined

In [6]:
# Define the function that evaluates the model on validation data
def evaluate_model(model, X_val, y_val):
    y_val_pred = model.predict(X_val)  # Predict on validation set
    f1 = f1_score(y_val, y_val_pred, average='macro')  # Calculate F1 (macro)
    accuracy = accuracy_score(y_val, y_val_pred)  # Calculate accuracy
    return f1, accuracy

In [7]:
max_features = 1006
ngram_range = (1, 2)

# Split data into training, validation and testing sets
X = df[['comment', 'word_count', 'char_count', 'avg_word_length']]
y = df['category']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42, stratify=y_temp)

### Vectorization and Resampling

In [ ]:
# Vectorize the data
X_train_combined, X_val_combined, X_test_combined = vectorize_data(X_train, X_val, X_test, max_features, ngram_range)

In [ ]:
# Apply resampling technique
X_resampled, y_resampled = ADASYN(random_state=42).fit_resample(X_train_combined, y_train)

### XGBoost

In [ ]:
# Define the Optuna objective function
def objective(trial):
    # Hyperparameters to optimize
    n_estimators = trial.suggest_int("n_estimators", 50, 500, step=10)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)

    # Initialize the XGBoost model with the suggested hyperparameters
    model = xgb.XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        objective='multi:softmax',
        num_class=3,  # Number of classes
        tree_method='hist',   # Use hist method for GPU
        device='cuda',        # Specify CUDA device for GPU training
        random_state=42,
    )

    # Fit the model on the resampled training data
    model.fit(X_resampled, y_resampled)

    # Evaluate the model on the validation set
    f1, accuracy = evaluate_model(model, X_val_combined, y_val)

    return accuracy, f1

In [ ]:
# Run Optuna optimization
study_xgb = optuna.create_study(directions=["maximize", "maximize"], study_name="XGBoost_Optimization")  # Multi-objective optimization for both F1 and accuracy
study_xgb.optimize(objective, n_trials=100)

[I 2024-10-23 07:32:58,321] A new study created in memory with name: XGBoost_Optimization
[I 2024-10-23 07:33:30,353] Trial 0 finished with values: [0.6193853427895981, 0.5696893450889243] and parameters: {'n_estimators': 420, 'max_depth': 6, 'learning_rate': 0.000981683522955723}.
[I 2024-10-23 07:35:32,469] Trial 1 finished with values: [0.7281323877068558, 0.6980806692126299] and parameters: {'n_estimators': 440, 'max_depth': 12, 'learning_rate': 0.006895094713133171}.
[I 2024-10-23 07:35:45,325] Trial 2 finished with values: [0.6253864338970722, 0.5822589971231921] and parameters: {'n_estimators': 210, 'max_depth': 5, 'learning_rate': 0.004823390227911081}.
[I 2024-10-23 07:36:34,716] Trial 3 finished with values: [0.6104746317512275, 0.5769254802345279] and parameters: {'n_estimators': 350, 'max_depth': 8, 'learning_rate': 0.00011252298249138059}.
[I 2024-10-23 07:37:53,359] Trial 4 finished with values: [0.6477541371158393, 0.6076811272388154] and parameters: {'n_estimators': 270

In [ ]:
best_trial = sorted(study_xgb.best_trials, key=lambda t: t.values[0], reverse=True)[0]

with mlflow.start_run() as run:
    mlflow.set_tag("mlflow.runName", "XGBoost")
    mlflow.set_tag("resampling_technique", "Adasyn")
    mlflow.set_tag("vectorizer_type", "TF-IDF")

    # Log best trial parameters
    mlflow.log_params(best_trial.params)

    # Log algorithm name as a parameter
    mlflow.log_param("algo_name", "XGBoost")

    # Extract parameters from the best trial
    best_params = best_trial.params

    # Initialize the model using the best trial parameters with unpacking (**)
    model = xgb.XGBClassifier(random_state=42, objective='multi:softmax',
                              num_class=3,  # Number of classes
                              tree_method='hist',   # Use hist method for GPU
                              device='cuda',        # Specify CUDA device for GPU training
                              **best_trial.params)

    # Train the model on the resampled training data
    model.fit(X_resampled, y_resampled)

    # Predictions on the test set
    y_test_pred = model.predict(X_test_combined)

    # Log classification metrics
    classification_rep = classification_report(y_test, y_test_pred, output_dict=True)
    accuracy = accuracy_score(y_test, y_test_pred)

    # Log accuracy
    mlflow.log_metric("accuracy", accuracy)

    # Log each metric from classification report
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Generate and log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - XGBoost")

    # Save and log the confusion matrix plot
    confusion_matrix_filename = "confusion_matrix.png"
    plt.savefig(confusion_matrix_filename)
    mlflow.log_artifact(confusion_matrix_filename)
    plt.close()

2024/10/23 10:22:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBoost at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/5/runs/ff3e2bc6b85c4886a6d4ed976fdff306.
2024/10/23 10:22:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/5.


### LightGBM

In [ ]:
import lightgbm as lgb

# Define the Optuna objective function for LightGBM
def objective_lgbm(trial):
    # Hyperparameters to optimize
    n_estimators = trial.suggest_int("n_estimators", 100, 450, step=10)
    max_depth = trial.suggest_int("max_depth", 5, 15)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)

    # Initialize the LightGBM model with GPU support
    model = lgb.LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        objective='multiclass',   # Multi-class classification objective
        num_class=3,              # Number of classes
        # device='gpu',             # Enable GPU
        random_state=42,
    )

    # Fit the model on the resampled training data
    model.fit(X_resampled, y_resampled)

    # Evaluate the model on the validation set
    f1, accuracy = evaluate_model(model, X_val_combined, y_val)

    return accuracy, f1

In [ ]:
# Run Optuna optimization
study_lgbm = optuna.create_study(directions=["maximize", "maximize"], study_name="LightGBM_Optimization")  # Multi-objective optimization for both F1 and accuracy
study_lgbm.optimize(objective_lgbm, n_trials=100)

[I 2024-10-23 11:11:35,863] A new study created in memory with name: LightGBM_Optimization


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.383046 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 111332
[LightGBM] [Info] Number of data points in the train set: 34692, number of used features: 1000
[LightGBM] [Info] Start training from score -1.096971
[LightGBM] [Info] Start training from score -1.123301
[LightGBM] [Info] Start training from score -1.076124
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[I 2024-10-23 11:12:09,725] Trial 0 finished with values: [0.7326786688488817, 0.7028695362028694] and parameters: {'n_estimators': 440, 'max_depth': 5, 'learning_rate': 0.014217094410984728}.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.236639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 111332
[LightGBM] [Info] Number of data points in the train set: 34692, number of used features: 1000
[LightGBM] [Info] Start training from score -1.096971
[LightGBM] [Info] Start training from score -1.123301
[LightGBM] [Info] Start training from score -1.076124
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[I 2024-10-23 11:12:25,738] Trial 1 finished with values: [0.6212038552464084, 0.5688100946895994] and parameters: {'n_estimators': 130, 'max_depth': 6, 'learning_rate': 0.0018749897417486567}.


...

[I 2024-10-23 12:09:29,081] Trial 97 finished with values: [0.8059647208583379, 0.7838448781902585] and parameters: {'n_estimators': 330, 'max_depth': 13, 'learning_rate': 0.04859813159579442}.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.376943 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 111332
[LightGBM] [Info] Number of data points in the train set: 34692, number of used features: 1000
[LightGBM] [Info] Start training from score -1.096971
[LightGBM] [Info] Start training from score -1.123301
[LightGBM] [Info] Start training from score -1.076124
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[I 2024-10-23 12:09:51,941] Trial 98 finished with values: [0.7497726859428987, 0.7229006664160093] and parameters: {'n_estimators': 320, 'max_depth': 5, 'learning_rate': 0.026677945624448245}.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.222219 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 111332
[LightGBM] [Info] Number of data points in the train set: 34692, number of used features: 1000
[LightGBM] [Info] Start training from score -1.096971
[LightGBM] [Info] Start training from score -1.123301
[LightGBM] [Info] Start training from score -1.076124


[I 2024-10-23 12:10:19,632] Trial 99 finished with values: [0.7650481905801054, 0.7421132561662986] and parameters: {'n_estimators': 220, 'max_depth': 10, 'learning_rate': 0.024876946733966055}.


In [ ]:
best_trial = sorted(study_lgbm.best_trials, key=lambda t: t.values[0], reverse=True)[0]

with mlflow.start_run() as run:
    mlflow.set_tag("mlflow.runName", "LightGBM")
    mlflow.set_tag("resampling_technique", "Adasyn")
    mlflow.set_tag("vectorizer_type", "TF-IDF")

    # Log best trial parameters
    mlflow.log_params(best_trial.params)

    # Log algorithm name as a parameter
    mlflow.log_param("algo_name", "LightGBM")

    # Extract parameters from the best trial
    best_params = best_trial.params

    # Initialize the model using the best trial parameters with unpacking (**)
    model = lgb.LGBMClassifier(
                              objective='multiclass',   # Multi-class classification objective
                              num_class=3,              # Number of classes
                              # device='gpu',             # Enable GPU
                              random_state=42, **best_trial.params)

    # Train the model on the resampled training data
    model.fit(X_resampled, y_resampled)

    # Predictions on the test set
    y_test_pred = model.predict(X_test_combined)

    # Log classification metrics
    classification_rep = classification_report(y_test, y_test_pred, output_dict=True)
    accuracy = accuracy_score(y_test, y_test_pred)

    # Log accuracy
    mlflow.log_metric("accuracy", accuracy)

    # Log each metric from classification report
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Generate and log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - LightGBM")

    # Save and log the confusion matrix plot
    confusion_matrix_filename = "confusion_matrix.png"
    plt.savefig(confusion_matrix_filename)
    mlflow.log_artifact(confusion_matrix_filename)
    plt.close()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.234523 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 111332
[LightGBM] [Info] Number of data points in the train set: 34692, number of used features: 1000
[LightGBM] [Info] Start training from score -1.096971
[LightGBM] [Info] Start training from score -1.123301
[LightGBM] [Info] Start training from score -1.076124
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

2024/10/23 12:13:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/5/runs/37e81da10a4c436092a2ef115f5bd6a3.
2024/10/23 12:13:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/5.


#### Without resampling, using class weights

In [11]:
import lightgbm as lgb

# Initialize the LightGBM model with GPU support
model = lgb.LGBMClassifier(
    n_estimators=390,
    max_depth=14,
    learning_rate=0.061795130706726105,
    objective='multiclass',   # Multi-class classification objective
    num_class=3,              # Number of classes
    # device='gpu',             # Enable GPU
    random_state=42,
    class_weight= 'balanced'
)

# Fit the model on the resampled training data
model.fit(X_train_combined, y_train)

# Evaluate the model on the validation set
f1, accuracy = evaluate_model(model, X_val_combined, y_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.318602 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78953
[LightGBM] [Info] Number of data points in the train set: 27496, number of used features: 998
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [12]:
f1, accuracy

(0.7818568603797807, 0.8014184397163121)

#### Tuning vectorization technique taking lightgbm as base learner

In [8]:
# Set or create an experiment
mlflow.set_experiment("Exp 2 - BoW vs TfIdf vs Word2vec")

<Experiment: artifact_location='mlflow-artifacts:/9a79f5eeed8641cb9572a547b3fbc09d', creation_time=1729440551320, experiment_id='2', last_update_time=1729440551320, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf vs Word2vec', tags={'mlflow.sharedViewState.00829c284b45d57939b76fd6febe9d7c1fcc3bbfb87f3c7ecf80063f898df0be': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["metrics.`accuracy`","attributes.`Source`","metrics.`-1_recall`","metrics.`-1_precision`","metrics.`0_recall`","metrics.`0_precision`","metrics.`1_recall`","metrics.`1_precision`"],"runsExpanded":{},"runsPinned":["21250fa2e78e4669b7ee15c4fe8d19d4"],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","viewMaximized":false,"runListHidden":false,"isAccordionReordered":false,"groupBy":"",

In [9]:
# Objective function for Optuna
def objective(trial):

    # Hyperparameters for TF-IDF Vectorizer
    max_features = trial.suggest_int("max_features", 1000, 10000, step=10)
    ngram_range = trial.suggest_categorical("ngram_range", ["(1, 2)", "(1, 3)"])
    ngram_range = eval(ngram_range)


    vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
    X_train_vec = vectorizer.fit_transform(X_train['comment']).toarray()
    X_val_vec = vectorizer.transform(X_val['comment']).toarray()

    # Combine additional features
    X_train_combined = np.hstack([X_train_vec, X_train[['word_count', 'char_count', 'avg_word_length']].values])
    X_val_combined = np.hstack([X_val_vec, X_val[['word_count', 'char_count', 'avg_word_length']].values])

    # Initialize the LightGBM model
    model = lgb.LGBMClassifier(
        n_estimators=390,
        max_depth=14,
        learning_rate=0.061795130706726105,
        objective='multiclass',   # Multi-class classification objective
        num_class=3,              # Number of classes
        random_state=42,
        class_weight= 'balanced'
        )
    model.fit(X_train_combined, y_train)

    y_val_pred = model.predict(X_val_combined)  # Predict on validation set
    f1 = f1_score(y_val, y_val_pred, average='macro')  # Calculate F1 (macro)
    accuracy = accuracy_score(y_val, y_val_pred)  # Calculate accuracy

    # Start a child MLflow run
    trial_name = f"{ngram_range}_{max_features}"
    with mlflow.start_run(nested=True, run_name=trial_name):
        # Log hyperparameters
        mlflow.log_params({
            "ngram_range": ngram_range,
            "max_features": max_features
        })

        mlflow.log_param("vectorizer_type", "TF-IDF")

        # Log model metrics
        mlflow.log_metric("accuracy", accuracy)

        # Logging the classification report
        classification_rep = classification_report(y_val, y_val_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):  # For precision, recall, f1-score, etc.
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

    return accuracy, f1

In [ ]:
# Run Optuna optimization
study = optuna.create_study(directions=["maximize", "maximize"], study_name="TFIDF_LightGBM_Optimization")
study.optimize(objective, n_trials=70)

In [19]:
mlflow.end_run()
# Log the parent MLflow run for the whole study
with mlflow.start_run(run_id='e73d97e3ac1c41048b1f68d9e06f1d18'):

    best_trial = sorted(study.best_trials, key=lambda t: t.values[0], reverse=True)[0]

    # Log best trial parameters
    mlflow.log_params(best_trial.params)

    # Log algorithm name as a parameter
    mlflow.log_param("algo_name", "LightGBM")

    # Extract parameters from the best trial
    best_params = best_trial.params

    vectorizer = TfidfVectorizer(ngram_range=eval(best_trial.params['ngram_range']), max_features=best_trial.params['max_features'])
    X_train_vec = vectorizer.fit_transform(X_train['comment']).toarray()
    X_test_vec = vectorizer.transform(X_test['comment']).toarray()

    # Combine additional features
    X_train_combined = np.hstack([X_train_vec, X_train[['word_count', 'char_count', 'avg_word_length']].values])
    X_test_combined = np.hstack([X_test_vec, X_test[['word_count', 'char_count', 'avg_word_length']].values])

    # Initialize the LightGBM model
    model = lgb.LGBMClassifier(
        n_estimators=390,
        max_depth=14,
        learning_rate=0.061795130706726105,
        objective='multiclass',   # Multi-class classification objective
        num_class=3,              # Number of classes
        random_state=42,
        class_weight='balanced'
        )

    model.fit(X_train_combined, y_train)

    # Predictions on the test set
    y_test_pred = model.predict(X_test_combined)

    # Log classification metrics
    classification_rep = classification_report(y_test, y_test_pred, output_dict=True)
    accuracy = accuracy_score(y_test, y_test_pred)

    # Log accuracy
    mlflow.log_metric("accuracy", accuracy)

    # Log each metric from classification report
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Generate and log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - LightGBM")

    # Save and log the confusion matrix plot
    confusion_matrix_filename = "confusion_matrix.png"
    plt.savefig(confusion_matrix_filename)
    mlflow.log_artifact(confusion_matrix_filename)
    plt.close()

2024/10/24 15:16:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run popular-sow-61 at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/2/runs/3247cbec947b454e9b0363b1b04d321b.
2024/10/24 15:16:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/2.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.833098 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 123708
[LightGBM] [Info] Number of data points in the train set: 27496, number of used features: 4170
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

2024/10/24 15:17:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run TFIDF_LightGBM_Optimization at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/2/runs/e73d97e3ac1c41048b1f68d9e06f1d18.
2024/10/24 15:17:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dakshvandanarathi/YT-Sentiment-Analyser.mlflow/#/experiments/2.


In [16]:
eval(best_trial.params['ngram_range'])

(1, 2)

In [20]:
sorted(study.best_trials, key=lambda t: t.values[0], reverse=True)[0]

FrozenTrial(number=68, state=TrialState.COMPLETE, values=[0.8597926895799236, 0.8475817525784809], datetime_start=datetime.datetime(2024, 10, 24, 14, 35, 2, 159732), datetime_complete=datetime.datetime(2024, 10, 24, 14, 36, 42, 287595), params={'max_features': 9410, 'ngram_range': '(1, 2)'}, user_attrs={}, system_attrs={'nsga2:generation': 1}, intermediate_values={}, distributions={'max_features': IntDistribution(high=10000, log=False, low=1000, step=10), 'ngram_range': CategoricalDistribution(choices=('(1, 2)', '(1, 3)'))}, trial_id=68, value=None)